In [ ]:
import pandas as pd
from py2neo import Graph
import matplotlib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics


In [ ]:
graph = Graph(host="localhost", auth=("neo4j", "test123"))

In [ ]:
query = "MATCH (clan:Clan)--(standing:War_Standing)-[r]-(war:War) MATCH (participant:War_Participant)--(standing) RETURN war.war_end_time as Time,clan.name as Clan, sum(participant.cards_earned) as cards, standing.participants as Participants, standing.wins as Wins, standing.battles_played as Played, r.standing as Result"

war_results = graph.run(query).to_data_frame()
war_results["average_cards"] = war_results["cards"] / war_results["Participants"]
war_results.set_index("Clan")

war_results.shape

In [ ]:
matplotlib.pyplot.scatter(war_results["average_cards"], war_results["Result"], )

In [ ]:
x_columns = [
    "cards",
    "Participants"
]

y_column = ["Result"]

X_data = war_results[x_columns]
y_data = war_results[y_column]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.33, random_state=42)

In [ ]:
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

In [ ]:
def scale_data(df, scaler):
    return pd.DataFrame(scaler.transform(df), columns = df.columns)

X_train_scaled = scale_data(X_train, X_scaler)
y_train_scaled = scale_data(y_train, y_scaler)
X_test_scaled = scale_data(X_test, X_scaler)
y_test_scaled = scale_data(y_test, y_scaler)

In [ ]:
matplotlib.pyplot.scatter(X_train_scaled["cards"] / X_train_scaled["Participants"], y_train_scaled["Result"])

In [ ]:
X_train_scaled.head()

In [ ]:
clf = svm.SVR()
clf.fit(X_train_scaled, y_train)

In [ ]:
y_train_prediction = clf.predict(X_train_scaled)
y_test_prediction = clf.predict(X_test_scaled)

mse_train = metrics.mean_squared_error(y_train, y_train_prediction)
mse_test = metrics.mean_squared_error(y_test, y_test_prediction)

print(mse_train)
print(mse_test)



In [ ]:
matplotlib.pyplot.scatter(y_train, y_train_prediction)